<img src="https://www.rp.edu.sg/images/default-source/default-album/rp-logo.png" width="200" alt="Republic Polytechnic"/>

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/koayst-rplesson/SDGAI_LLMforGenAIApp_Labs/blob/main/L14/L14.ipynb)

# Setup and Installation

You can run this Jupyter notebook either on your local machine or run it at Google Colab.

* For local machine, it is recommended to install Anaconda and create a new development environment called `c3669c`.
* Pip/Conda install the libraries stated below when necessary.
---

# <font color='red'>ATTENTION</font>

## Google Colab
- If you are running this code in Google Colab, **DO NOT** store the API Key in a text file and load the key later from Google Drive. This is insecure and will expose the key.
- **DO NOT** hard code the API Key directly in the Python code, even though it might seem convenient for quick development.
- You need to enter the API key at python code `getpass.getpass()` when ask.

## Local Environment/Laptop
- If you are running this code locally in your laptop, you can create a env.txt and store the API key there.
- Make sure env.txt is in the same directory of this Jupyter notebook.
- You need to install `python-dotenv` and run the Python code to load in the API key.

---
```
%pip install python-dotenv

from dotenv import load_dotenv

load_dotenv('env.tx')
openai_api_key = os.getenv('OPENAI_API_KEY')
```
---

## GitHub/GitLab
- **DO NOT** `commit` or `push` API Key to services like GitHub or GitLab.

# Lesson 14

The code below demonstrates the LangSmith function using a simple Retrieval-Augmented Generation (RAG) system. It begins by building a vector store through the retrieval of content from a website's sitemap, such as https://docs.smith.langchain.com/sitemap.xml. A sitemap is a structured file or page that provides detailed information about the content of a website, enabling search engines like Google to understand its structure more effectively.

This process constitutes the "retrieval" phase of a RAG system. The retrieved content is chunked into smaller segments and stored in a directory named `chroma_langsmith_db`. The next step involves answer generation. The RAG system leverages the LangChain Expression Language (LCEL) to accomplish this.

As the Python code executes, observe that the response output is printed. However, if you are using LangSmith, manual output printing for tracing or debugging purposes is unnecessary, as LangSmith automatically facilitates these functionalities.

In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain
%pip install --quiet -U langchain-openai
%pip install --quiet -U langchain-community
%pip install --quiet -U langsmith
%pip install --quiet -U "langchain-chroma>=0.1.2"

In [ ]:
# langchain              0.3.11
# langgraph              0.2.59
# langchain-core         0.3.24
# langchain-openai       0.2.12
# langchain-community    0.3.12
# openai                 1.57.2
# pydantic               2.10.3
# tavily-python          0.5.0
# wikipedia              1.4.0
# chroma                 0.5.23
# PyMuPDF                1.25.1

In [1]:
import getpass
import os

In [2]:
# setup the OpenAI API Key

# get OpenAI API key ready and enter it when ask
os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [3]:
# setup the LangChain LangSmith API Key

# get OpenAI API key ready and enter it when ask
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"]= "LangSmith-Lesson-14"

 ········


In [4]:
# the python code below is to stop the warning message from showing up
# USER_AGENT environment variable not set, consider setting it to identify your requests.
os.environ['USER_AGENT'] = "my_user_agent"

---

In [5]:
from langchain_community.document_loaders.sitemap import SitemapLoader

from langchain_chroma import Chroma

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

from langsmith import traceable

from pydantic import BaseModel, Field
from typing import List, Optional

import nest_asyncio

In [6]:
# set up the model

llm_model = ChatOpenAI(
    model = 'gpt-4o-mini',
)

In [7]:
# the below python code is needed or you will get error:
# asyncio.run() cannot be called from a running event loop

nest_asyncio.apply()

In [8]:
# set up the vector store

chroma_path = "chroma"
collection_name = "rag-chroma-langsmith"

@traceable(run_type="chain")
def get_vector_db_retriever():

    persist_directory = "./chroma_langsmith_db"
    embeddings = OpenAIEmbeddings()
    
    if os.path.exists(persist_directory):
        vector_store = Chroma(
            persist_directory = persist_directory,
            embedding_function = embeddings,
            collection_name = collection_name
        )

        return vector_store.as_retriever()
    
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml")
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, 
        chunk_overlap=100
    )

    texts = text_splitter.split_documents(ls_docs)

    vector_store = Chroma.from_documents(
        documents = texts, 
        embedding=embeddings,
        persist_directory = persist_directory,
        collection_name = collection_name
    )

    print("Vector store created")

    return vector_store.as_retriever()       

In [9]:
# it takes some time to create the vector store if you running this first time
# else the vector store is already created, it will load from existing vector db

retriever = get_vector_db_retriever()

Failed to use model_dump to serialize <class 'langchain_core.vectorstores.base.VectorStoreRetriever'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'langchain_chroma.vectorstores.Chroma'>)


In [10]:
# test retrieval by invoking it
vector_db_response = retriever.invoke("What is LangSmith used for?")

# print the contexts retrieved
docs = ""
for index, doc in enumerate(vector_db_response):
    docs = f'{docs}\n{index}\n--------------\n{doc.page_content}\n--------------\n'

In [11]:
# the output below is meant for observation and comparison later

print(docs)


0
--------------
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:
--------------

1
--------------
LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.
--------------

2
--------------
LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
LangSmith is a standalone pla

In [12]:
# set up prompt template

rag_system_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

human_prompt="""
{formatted_docs}


Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", rag_system_prompt),
    ("human", human_prompt)
])

In [13]:
# set up the chain using LCEL

rag_chain = rag_prompt | llm_model | StrOutputParser()

In [14]:
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    
    response = rag_chain.invoke({
        "formatted_docs" : formatted_docs,
        "question" : question
    })

    print(f"1. generate_response: formatted_docs\n{formatted_docs}")
    print("-"*10)
    print(f"2. generate_response: question\n{question}")
    print("-"*10)
    print(f"3. generate_response: responese\n{response}")        
    
    return response

---

In [15]:
question = "What is LangSmith used for?"

response = generate_response(question, vector_db_response)

1. generate_response: formatted_docs
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
LangSmith is a standalone platform that can be used on it's own no matter 

In [16]:
response

'LangSmith is used for building production-grade LLM applications, enabling users to monitor, evaluate, and test their applications effectively. It supports various workflows throughout the application development lifecycle. Additionally, LangSmith integrates with LangChain and LangGraph, but can also be used as a standalone platform.'

---